In [1]:
from utilities.data_loader import fetch_forex_data, fetch_stock_data
fetch_forex_data('USD','CNY').to_csv("historical_data_usd_cny.csv")
fetch_forex_data('AUD','JPY').to_csv("historical_data_aud_jpy.csv")
fetch_stock_data('TTE').to_csv("historical_data_total.csv")
fetch_stock_data('SAP').to_csv("historical_data_sap.csv")
fetch_stock_data('NFLX').to_csv("historical_data_netflix.csv")
fetch_stock_data('BHP').to_csv("historical_data_bhp.csv")
fetch_stock_data('TSL').to_csv("historical_data_tesla.csv")

In [2]:
import pandas as pd

In [3]:
total_df = pd.read_csv("data/historical_data_total.csv")

In [26]:
# Assurez-vous que la colonne "date" est bien au format datetime
total_df["date"] = pd.to_datetime(total_df["date"])

# Trouver la date limite (10 ans avant la date max)
date_limite = total_df["date"].max() - pd.DateOffset(years=5)

# Filtrer le DataFrame pour ne garder que les données des 10 dernières années
df_recent = total_df[total_df["date"] >= date_limite]

In [54]:
import plotly.graph_objects as go
import numpy as np
from scipy.stats import norm

# 📌 Suppression des NaN
data = df_recent["simple_return"].dropna()

# Calcul de la moyenne (μ) et de l'écart-type (σ)
mean = data.mean()
std = data.std()

# 🔲 Création de l'histogramme avec contours bien visibles
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=data,
    nbinsx=300,  # Ajuste le nombre de bins pour plus de lisibilité
    marker=dict(
        color="royalblue",
        line=dict(width=1.2, color="black")
    ),
    opacity=0.75,
    showlegend=False  # 🔥 Supprime la légende de l'histogramme
))

# 📈 Création de la courbe gaussienne avec scipy.stats.norm
x_values = np.linspace(data.min(), data.max(), 1000)
pdf = norm.pdf(x_values, loc=mean, scale=std)
bin_width = (data.max() - data.min()) / 300  # largeur d'un bin
scaled_pdf = pdf * len(data) * bin_width

fig.add_trace(go.Scatter(
    x=x_values,
    y=scaled_pdf,
    mode='lines',
    line=dict(color='red', width=2),
    name=f'Gaussienne (μ={mean:.2%}, σ={std:.2%})'
))

# ➖ Ajout d'une ligne verticale pour μ en vert fluo (lime)
fig.add_shape(
    type="line",
    x0=mean, x1=mean,
    y0=0, y1=max(scaled_pdf),
    line=dict(color="lime", dash="dash")
)
fig.add_annotation(
    x=mean,
    y=max(scaled_pdf)*0.9,
    text=f"μ: {mean:.2%}",
    showarrow=True,
    arrowhead=2,
    ax=40,
    ay=-40,
    font=dict(color="lime")
)

# ➖ Ajout des lignes verticales pour μ - σ et μ + σ en orange (pointillé)
fig.add_shape(
    type="line",
    x0=mean - std, x1=mean - std,
    y0=0, y1=max(scaled_pdf),
    line=dict(color="orange", dash="dot")
)
fig.add_annotation(
    x=mean - std,
    y=max(scaled_pdf)*0.7,
    text=f"μ - σ: {(mean - std):.2%}",
    showarrow=True,
    arrowhead=2,
    ax=-40,
    ay=-40,
    font=dict(color="orange")
)

fig.add_shape(
    type="line",
    x0=mean + std, x1=mean + std,
    y0=0, y1=max(scaled_pdf),
    line=dict(color="orange", dash="dot")
)
fig.add_annotation(
    x=mean + std,
    y=max(scaled_pdf)*0.7,
    text=f"μ + σ: {(mean + std):.2%}",
    showarrow=True,
    arrowhead=2,
    ax=40,
    ay=-40,
    font=dict(color="orange")
)

# 🎨 Mise en forme du graphique avec légende légèrement décalée à gauche sous le titre
fig.update_layout(
    title="Distribution des Rendements Quotidiens",
    xaxis_title="Rendement Quotidien",
    yaxis_title="Nombre de Jours",
    xaxis=dict(tickformat=".2%"),
    template="plotly_white",
    legend=dict(
        orientation="h",    # Orientation horizontale
        yanchor="bottom",
        y=0.93,             # Ajustez cette valeur pour positionner la légende juste sous le titre
        xanchor="left",     # Ancre la légende à gauche
        x=0.1               # Décale légèrement vers la gauche
    )
)

# 📈 Affichage
fig.show()


In [65]:
import plotly.graph_objects as go
import numpy as np
from scipy.stats import norm

def plot_return_distribution(df, nbins=300, duration=None, return_type='simple'):
    """
    Affiche la distribution des rendements historiques sous forme d'histogramme 
    avec la courbe gaussienne associée, et ajoute des annotations pour la moyenne et l'écart-type.
    
    Paramètres:
        df (DataFrame): DataFrame contenant les colonnes "simple_return" et "log_return".
        nbins (int): Nombre de bins pour l'histogramme.
        duration (int): Nombre de dernières observations à considérer. Si None, tous les rendements sont pris en compte.
        return_type (str): 'simple' pour utiliser la colonne "simple_return" ou 'log' pour "log_return".
    
    Retour:
        fig (Figure): Objet Figure de Plotly contenant le graphique.
    """
    # Sélection de la série de rendement selon le type choisi
    if return_type == 'simple':
        data = df["simple_return"].dropna()
    elif return_type == 'log':
        data = df["log_return"].dropna()
    else:
        raise ValueError("Le paramètre return_type doit être 'simple' ou 'log'.")

    # Filtrage de la série selon la durée (nombre de dernières observations)
    if duration is not None:
        data = data.tail(duration)
    
    # Calcul de la moyenne (μ) et de l'écart-type (σ)
    mean = data.mean()
    std = data.std()

    # Création de la figure Plotly
    fig = go.Figure()

    # Ajout de l'histogramme
    fig.add_trace(go.Histogram(
        x=data,
        nbinsx=nbins,
        marker=dict(
            color="royalblue",
            line=dict(width=1.2, color="black")
        ),
        opacity=0.75,
        showlegend=False
    ))

    # Calcul et ajout de la courbe gaussienne
    x_values = np.linspace(data.min(), data.max(), 1000)
    pdf = norm.pdf(x_values, loc=mean, scale=std)
    bin_width = (data.max() - data.min()) / nbins
    scaled_pdf = pdf * len(data) * bin_width

    fig.add_trace(go.Scatter(
        x=x_values,
        y=scaled_pdf,
        mode='lines',
        line=dict(color='red', width=2),
        name=f'Gaussienne (μ={mean:.2%}, σ={std:.2%})'
    ))

    # Ligne verticale pour la moyenne (μ)
    fig.add_shape(
        type="line",
        x0=mean, x1=mean,
        y0=0, y1=max(scaled_pdf),
        line=dict(color="lime", dash="dash")
    )
    fig.add_annotation(
        x=mean,
        y=max(scaled_pdf)*0.9,
        text=f"μ: {mean:.2%}",
        showarrow=True,
        arrowhead=2,
        ax=40,
        ay=-40,
        font=dict(color="lime")
    )

    # Lignes verticales pour μ - σ et μ + σ
    fig.add_shape(
        type="line",
        x0=mean - std, x1=mean - std,
        y0=0, y1=max(scaled_pdf),
        line=dict(color="orange", dash="dot")
    )
    fig.add_annotation(
        x=mean - std,
        y=max(scaled_pdf)*0.7,
        text=f"μ - σ: {(mean - std):.2%}",
        showarrow=True,
        arrowhead=2,
        ax=-40,
        ay=-40,
        font=dict(color="orange")
    )

    fig.add_shape(
        type="line",
        x0=mean + std, x1=mean + std,
        y0=0, y1=max(scaled_pdf),
        line=dict(color="orange", dash="dot")
    )
    fig.add_annotation(
        x=mean + std,
        y=max(scaled_pdf)*0.7,
        text=f"μ + σ: {(mean + std):.2%}",
        showarrow=True,
        arrowhead=2,
        ax=40,
        ay=-40,
        font=dict(color="orange")
    )

    # Mise en forme du graphique
    fig.update_layout(
        title="Distribution des Rendements Quotidiens",
        xaxis_title="Rendement Quotidien",
        yaxis_title="Nombre de Jours",
        xaxis=dict(tickformat=".2%"),
        template="plotly_white",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=0.93,
            xanchor="left",
            x=0.1
        )
    )

    return fig

In [66]:
plot_return_distribution(df_recent, nbins=300, duration=1000, return_type='log')

In [2]:
import yfinance as yf

# Création de l'objet ticker pour ^TNX
ticker = yf.Ticker("^TNX")

# Récupération des données sur 5 ans
data = ticker.history(period="5y")

In [4]:
import yfinance as yf

# Création de l'objet ticker pour Apple
ticker = yf.Ticker("AAPL")

# Récupération des dates d'expiration disponibles
options_dates = ticker.options
print("Dates d'expiration disponibles :", options_dates)

# Récupération de la chaîne d'options pour la première date d'expiration
if options_dates:
    option_chain = ticker.option_chain(options_dates[0])
    print("Calls :", option_chain.calls)
    print("Puts :", option_chain.puts)


Dates d'expiration disponibles : ('2025-02-28', '2025-03-07', '2025-03-14', '2025-03-21', '2025-03-28', '2025-04-04', '2025-04-17', '2025-05-16', '2025-06-20', '2025-07-18', '2025-08-15', '2025-09-19', '2025-10-17', '2025-12-19', '2026-01-16', '2026-03-20', '2026-06-18', '2026-12-18', '2027-01-15', '2027-06-17')
Calls :          contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0   AAPL250228C00110000 2025-01-21 19:20:02+00:00   110.0     111.67  133.80   
1   AAPL250228C00140000 2025-02-20 14:59:32+00:00   140.0     105.09  103.80   
2   AAPL250228C00145000 2025-02-11 15:05:42+00:00   145.0      86.52   98.85   
3   AAPL250228C00150000 2025-02-18 15:36:57+00:00   150.0      94.39   94.75   
4   AAPL250228C00165000 2025-02-21 18:26:50+00:00   165.0      82.75   79.70   
5   AAPL250228C00170000 2025-02-20 19:37:20+00:00   170.0      75.30   74.75   
6   AAPL250228C00175000 2025-02-21 18:21:14+00:00   175.0      72.50   70.35   
7   AAPL250228C00180000 2025-02-21 19:

In [3]:
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-02-21 00:00:00-06:00,1.485,1.492,1.439,1.471,0,0.0,0.0
2020-02-24 00:00:00-06:00,1.396,1.402,1.352,1.377,0,0.0,0.0
2020-02-25 00:00:00-06:00,1.374,1.377,1.307,1.330,0,0.0,0.0
2020-02-26 00:00:00-06:00,1.369,1.381,1.302,1.310,0,0.0,0.0
2020-02-27 00:00:00-06:00,1.272,1.327,1.243,1.299,0,0.0,0.0
...,...,...,...,...,...,...,...
2025-02-14 00:00:00-06:00,4.531,4.533,4.447,4.472,0,0.0,0.0
2025-02-18 00:00:00-06:00,4.519,4.546,4.507,4.544,0,0.0,0.0
2025-02-19 00:00:00-06:00,4.566,4.576,4.529,4.535,0,0.0,0.0


In [5]:
import sys
print(sys.path)

['/Users/loubrunet/miniconda3/envs/var_project/lib/python312.zip', '/Users/loubrunet/miniconda3/envs/var_project/lib/python3.12', '/Users/loubrunet/miniconda3/envs/var_project/lib/python3.12/lib-dynload', '', '/Users/loubrunet/.local/lib/python3.12/site-packages', '/Users/loubrunet/miniconda3/envs/var_project/lib/python3.12/site-packages']


In [6]:
import streamlit

In [7]:
def get_fundamental_info(ticker_symbol):
    """
    Récupère les informations fondamentales d'une entreprise à partir de son ticker.

    Args:
        ticker_symbol (str): Le symbole boursier de l'entreprise (ex: "AAPL" pour Apple).

    Returns:
        dict: Un dictionnaire contenant le nom de l'entreprise, le secteur, l'industrie,
              la capitalisation boursière et un résumé de l'activité.
    """
    ticker = yf.Ticker(ticker_symbol)
    info = ticker.info
    
    return {
        "Nom de l'entreprise": info.get("longName"),
        "Secteur": info.get("sector"),
        "Industrie": info.get("industry"),
        "Capitalisation boursière": info.get("marketCap"),
        "Résumé de l'activité": info.get("longBusinessSummary")
    }

In [1]:
get_fundamental_info('THLLY')

NameError: name 'get_fundamental_info' is not defined

In [2]:
from data.data_loader import tickerf

In [3]:
df = tickerf('TSLA')

In [5]:
df.reset_index()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-02-24 00:00:00-05:00,55.933334,57.566666,54.813332,55.585999,227883000,0.0,0.0
1,2020-02-25 00:00:00-05:00,56.599998,57.106667,52.466667,53.327332,259357500,0.0,0.0
2,2020-02-26 00:00:00-05:00,52.166668,54.220669,51.740665,51.919998,211282500,0.0,0.0
3,2020-02-27 00:00:00-05:00,48.666668,49.318001,44.599998,45.266666,364158000,0.0,0.0
4,2020-02-28 00:00:00-05:00,41.980000,46.034668,40.768002,44.532665,363343500,0.0,0.0
...,...,...,...,...,...,...,...,...
1252,2025-02-14 00:00:00-05:00,360.619995,362.000000,347.500000,355.839996,68277300,0.0,0.0
1253,2025-02-18 00:00:00-05:00,355.010010,359.100006,350.019989,354.109985,51631700,0.0,0.0
1254,2025-02-19 00:00:00-05:00,354.000000,367.339996,353.670013,360.559998,67094400,0.0,0.0
1255,2025-02-20 00:00:00-05:00,361.510010,362.299988,348.000000,354.399994,45965400,0.0,0.0


In [12]:
import pandas as pd
import numpy as np

def compute_returns(df):
    """
    Calcule les retours quotidiens simples et logarithmiques à partir de la colonne 'Close'
    du DataFrame. Le DataFrame est supposé contenir une colonne 'Date' (de type datetime ou string)
    et une colonne 'Close'.
    
    Paramètres:
        df (pd.DataFrame): DataFrame contenant au minimum les colonnes 'Date' et 'Close'.
        
    Retour:
        pd.DataFrame: DataFrame d'origine avec deux nouvelles colonnes:
                      - 'Simple Return' : Retour simple calculé comme (Close_i / Close_{i-1}) - 1
                      - 'Log Return'    : Retour logarithmique calculé comme ln(Close_i / Close_{i-1})
    """
    # On s'assure que le DataFrame est trié par date
    df = df.sort_values('Date').reset_index(drop=True)
    
    # Calcul du retour simple
    df['simple_return'] = df['Close'].pct_change()
    
    # Calcul du retour logarithmique
    df['log_return'] = np.log(df['Close'] / df['Close'].shift(1))
    
    return df.iloc[1:]


In [13]:
compute_returns(df.reset_index())

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,simple_return,log_return
1,2020-02-25 00:00:00-05:00,56.599998,57.106667,52.466667,53.327332,259357500,0.0,0.0,-0.040634,-0.041482
2,2020-02-26 00:00:00-05:00,52.166668,54.220669,51.740665,51.919998,211282500,0.0,0.0,-0.026390,-0.026745
3,2020-02-27 00:00:00-05:00,48.666668,49.318001,44.599998,45.266666,364158000,0.0,0.0,-0.128146,-0.137133
4,2020-02-28 00:00:00-05:00,41.980000,46.034668,40.768002,44.532665,363343500,0.0,0.0,-0.016215,-0.016348
5,2020-03-02 00:00:00-05:00,47.417332,49.579334,45.778000,49.574669,302925000,0.0,0.0,0.113220,0.107257
...,...,...,...,...,...,...,...,...,...,...
1252,2025-02-14 00:00:00-05:00,360.619995,362.000000,347.500000,355.839996,68277300,0.0,0.0,-0.000281,-0.000281
1253,2025-02-18 00:00:00-05:00,355.010010,359.100006,350.019989,354.109985,51631700,0.0,0.0,-0.004862,-0.004874
1254,2025-02-19 00:00:00-05:00,354.000000,367.339996,353.670013,360.559998,67094400,0.0,0.0,0.018215,0.018051
1255,2025-02-20 00:00:00-05:00,361.510010,362.299988,348.000000,354.399994,45965400,0.0,0.0,-0.017085,-0.017232


In [14]:
from utilities.graphs_plots import plot_return_distribution

In [15]:
plot_return_distribution(compute_returns(df.reset_index()))